In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = fasterrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

model_path = '../Models/best_model_fasterrcnn.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

In [2]:
import torchvision

def apply_nms(boxes, scores, iou_threshold=0.5):
    keep_indices = torchvision.ops.nms(boxes, scores, iou_threshold)
    return keep_indices.cpu().numpy()

In [3]:
import torchvision.transforms as T
from PIL import Image, ImageDraw

def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    image_tensor = transform(image)
    return image, image_tensor

In [4]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_prediction(image, boxes, scores, labels, threshold=0.5):
    draw_image = image.copy()
    draw = ImageDraw.Draw(draw_image)
    
    filtered_boxes = []
    filtered_scores = []
    filtered_labels = []
    
    for box, score, label in zip(boxes, scores, labels):
        if score >= threshold:
            filtered_boxes.append(box)
            filtered_scores.append(score)
            filtered_labels.append(label)
    
    for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
        x1, y1, x2, y2 = box
        
        draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=3)
        
        label_text = f"Pistola: {score:.2f}"
        draw.text((x1, y1-15), label_text, fill="red")
    
    plt.figure(figsize=(12, 8))
    plt.imshow(np.array(draw_image))
    plt.axis('off')
    plt.show()
    
    return draw_image

In [5]:
def detect_guns(image_path, confidence_threshold=0.5):
    original_image, image_tensor = load_image(image_path)
    
    with torch.no_grad():
        prediction = model([image_tensor.to(device)])
    
    boxes = prediction[0]['boxes'].cpu().numpy()
    scores = prediction[0]['scores'].cpu().numpy()
    labels = prediction[0]['labels'].cpu().numpy()
    
    keep_indices = apply_nms(torch.tensor(boxes), torch.tensor(scores), iou_threshold=0.2)
    
    boxes = boxes[keep_indices]
    scores = scores[keep_indices]
    labels = labels[keep_indices]
    
    result_image = visualize_prediction(original_image, boxes, scores, labels, confidence_threshold)
    
    return {
        'boxes': boxes,
        'scores': scores,
        'labels': labels,
        'result_image': result_image
    }

In [ ]:
if __name__ == "__main__":
    test_images = os.listdir("../Dataset/fasterrcnn/test")
    test_images = [os.path.join("../Dataset/fasterrcnn/test", image) for image in test_images]
    guns_detected = 0
    
    for image_path in test_images:
        print(f"Esecuzione rilevamento armi su: {image_path}")
        results = detect_guns(image_path, confidence_threshold=0.6)
        guns_detected += sum(1 for score in results['scores'] if score >= 0.6)
        
    print(f"Armi rilevate: {guns_detected}")